In [3]:
import os

In [5]:
fs = os.listdir()
fs = [x for x in fs if x.startswith('slurm')]
for x in fs:
    os.remove(x)

In [ ]:
for dt in ['hESC', 'hHep', 'mDC', 'mESC', 'mHSC-E', 'mHSC-GM', 'mHSC-L']:
    for _ in range(10):
        os.system(f'sbatch recomb_benchmark.sh grnboost2 {dt}')

In [ ]:
for dt in ['hESC', 'hHep', 'mDC', 'mESC', 'mHSC-E', 'mHSC-GM', 'mHSC-L']:
    for _ in range(10):
        os.system(f'sbatch recomb_benchmark.sh genie3 {dt}')

In [ ]:
for _ in range(10):
    os.system(f'sbatch recomb_benchmark_gpu.sh deepsem')

In [ ]:
for _ in range(10):
    os.system(f'sbatch recomb_benchmark_gpu.sh deepseme')

In [ ]:
for _ in range(10):
    os.system(f'sbatch recomb_benchmark_gpu.sh dazzle')

In [ ]:
for _ in range(10):
    os.system(f'sbatch recomb_benchmark_gpu.sh regdiffusion')

In [ ]:
for _ in range(10):
    os.system(f'sbatch recomb_benchmark_gpu.sh regdiffusionda')

In [1]:
!sbatch recomb_benchmark_reduced.sh genie3 0.9
!sbatch recomb_benchmark_reduced.sh genie3 0.8
!sbatch recomb_benchmark_reduced.sh genie3 0.5
!sbatch recomb_benchmark_reduced.sh genie3 0.2
!sbatch recomb_benchmark_reduced.sh genie3 0.1

Submitted batch job 44842720
Submitted batch job 44842721
Submitted batch job 44842722
Submitted batch job 44842723
Submitted batch job 44842724


In [4]:
for train_split in [0.9, 0.8, 0.5, 0.2, 0.1]:
    os.system(f'sbatch recomb_benchmark_reduced.sh grnboost2 {train_split}')

Submitted batch job 44842726
Submitted batch job 44842727
Submitted batch job 44842728
Submitted batch job 44842729
Submitted batch job 44842730


In [2]:
import numpy as np
from regdiffusion import runRegDiffusion, DEFAULT_REGDIFFUSION_CONFIGS
from dazzle import load_beeline, extract_edges, get_metrics, DAZZLE, LightLogger, runDAZZLE, runDAZZLE_ensemble, DEFAULT_DAZZLE_CONFIGS, DEFAULT_DEEPSEM_CONFIGS
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
configs = DEFAULT_REGDIFFUSION_CONFIGS
configs['verbose'] = False
ncell = 1000
time_log = []
for m in ['deepsem*', 'regdiffusion']:
    for ngene in [100, 500, 1000, 2000, 5000, 10000]:
        start = datetime.now()
        fake_data = np.random.randn(ncell, ngene)
        if m == 'regdiffusion':
            model = runRegDiffusion(fake_data, configs)
        else:
            model = runDAZZLE(fake_data, DEFAULT_DEEPSEM_CONFIGS)
        time_cost = (datetime.now() - start).total_seconds()
        print(f'Time cost: {time_cost}')
        time_log.append({
            'n_cell': ncell, 'n_gene': ngene, 'model': m, 
            'time_cost': time_cost
        })

100%|██████████| 120/120 [00:08<00:00, 14.29it/s]


Time cost: 8.417756


100%|██████████| 120/120 [00:13<00:00,  9.20it/s]


Time cost: 13.087809


100%|██████████| 120/120 [00:23<00:00,  5.22it/s]


...

Time cost: 14.752962


100%|██████████| 250/250 [00:57<00:00,  4.36it/s]


Time cost: 57.972512


100%|██████████| 250/250 [03:08<00:00,  1.32it/s]

Time cost: 190.665432


In [11]:
pd.DataFrame(time_log).to_csv('time_cost.csv', index=False)

In [3]:
dt = 'mESC'
bl_data, bl_gt = load_beeline(benchmark_data=dt, benchmark_setting='1000_STRING')

In [4]:
configs = DEFAULT_REGDIFFUSION_CONFIGS
configs['train_split'] = 1.0

In [7]:
configs

{'T': 5000,
 'start_noise': 0.0001,
 'end_noise': 0.02,
 'lr_nn': 0.001,
 'lr_adj': 2e-05,
 'weight_decay_nn': 0.1,
 'weight_decay_adj': 0.01,
 'sparse_loss_coef': 0.25,
 'adj_dropout': 0.3,
 'batch_size': 128,
 'time_dim': 64,
 'celltype_dim': 32,
 'hidden_dims': [16, 16, 16],
 'n_epoch': 250,
 'device': 'cuda',
 'verbose': False,
 'train_split': 1.0,
 'train_split_seed': 123,
 'eval_on_n_steps': 10}

In [6]:
0.02 / 10

0.002

In [9]:
configs['start_noise'] = 0.0021
regdiff = runRegDiffusion(bl_data.X, configs, ground_truth=bl_gt)
metrics = get_metrics(regdiff.get_adj(), bl_gt)
print(metrics)

100%|██████████| 250/250 [00:10<00:00, 23.98it/s]


{'AUPR': 0.05200386121239633, 'AUPRR': 2.4453345287467423, 'EP': 776, 'EPR': 4.303479868212178}


In [11]:
configs['start_noise'] = 0.0001
regdiff = runRegDiffusion(bl_data.X, configs, ground_truth=bl_gt)
metrics = get_metrics(regdiff.get_adj(), bl_gt)
print(metrics)

100%|██████████| 250/250 [00:10<00:00, 24.93it/s]


{'AUPR': 0.051378516993993234, 'AUPRR': 2.4159294850833937, 'EP': 752, 'EPR': 4.170382552700461}


In [6]:
configs = DEFAULT_REGDIFFUSION_CONFIGS
configs['train_split'] = 1.0

logger = LightLogger()
final_metrics = []
for hd in [16, 32, 64, 128, 256]:
    for _ in range(10):
        configs['hidden_dims'] = [hd, hd, hd]
        start_time = datetime.now()
        regdiff = runRegDiffusion(bl_data.X, configs, ground_truth=bl_gt, logger=logger)
        metrics = get_metrics(regdiff.get_adj(), bl_gt)
        metrics['hidden_dims'] = hd
        metrics['time_cost'] = datetime.now() - start_time
        final_metrics.append(metrics)

100%|██████████| 250/250 [00:52<00:00,  4.73it/s]


In [8]:
for hd in [8]:
    for _ in range(10):
        configs['hidden_dims'] = [hd, hd, hd]
        start_time = datetime.now()
        regdiff = runRegDiffusion(bl_data.X, configs, ground_truth=bl_gt, logger=logger)
        metrics = get_metrics(regdiff.get_adj(), bl_gt)
        metrics['hidden_dims'] = hd
        metrics['time_cost'] = datetime.now() - start_time
        final_metrics.append(metrics)

100%|██████████| 250/250 [00:06<00:00, 39.75it/s]


In [9]:
logger.to_df().to_csv('results/hidden_dim.csv', index=False)

In [6]:
final_metrics

[{'AUPR': 0.05174711296582358,
  'AUPRR': 2.433261668426327,
  'EP': 754,
  'EPR': 4.181473995659771,
  'hidden_dims': 16,
  'time_cost': datetime.timedelta(seconds=9, microseconds=859782)},
 {'AUPR': 0.05194929324011775,
  'AUPRR': 2.442768624145322,
  'EP': 760,
  'EPR': 4.2147483245377,
  'hidden_dims': 32,
  'time_cost': datetime.timedelta(seconds=9, microseconds=46477)},
 {'AUPR': 0.05133098109170356,
  'AUPRR': 2.4136942436895037,
  'EP': 754,
  'EPR': 4.181473995659771,
  'hidden_dims': 64,
  'time_cost': datetime.timedelta(seconds=14, microseconds=515699)},
 {'AUPR': 0.05159759438036729,
  'AUPRR': 2.4262309797201107,
  'EP': 762,
  'EPR': 4.22583976749701,
  'hidden_dims': 128,
  'time_cost': datetime.timedelta(seconds=23, microseconds=720168)},
 {'AUPR': 0.050639155669202365,
  'AUPRR': 2.3811631093839667,
  'EP': 753,
  'EPR': 4.175928274180116,
  'hidden_dims': 256,
  'time_cost': datetime.timedelta(seconds=49, microseconds=73753)}]

In [5]:
train_split = 0.5
random_state = np.random.RandomState(123)
train_test_split = random_state.rand(bl_dt.X.shape[0])
train_index = train_test_split <= train_split
test_index = train_test_split > train_split

dazzle_model, adjs = runDAZZLE(bl_dt.X[train_index, :], DEFAULT_DAZZLE_CONFIGS)

100%|█████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:11<00:00, 10.60it/s]


In [6]:
get_metrics(dazzle_model.get_adj(), bl_gt)

{'AUPR': 0.044493309462890374,
 'AUPRR': 2.092172069367125,
 'EP': 629,
 'EPR': 3.4882588107029124}

In [8]:
regdiff_model = runRegDiffusion(bl_dt.X[train_index, :], DEFAULT_REGDIFFUSION_CONFIGS)
get_metrics(regdiff_model.get_adj(), bl_gt)

  8%|██████▉                                                                               | 20/250 [00:00<00:02, 95.90it/s]

{'train_loss': 0.8930535316467285}
{'train_loss': 0.829348087310791}
{'train_loss': 0.7519440650939941}


 24%|████████████████████▋                                                                | 61/250 [00:00<00:01, 127.68it/s]

{'train_loss': 0.6766452789306641}
{'train_loss': 0.5885483622550964}
{'train_loss': 0.5562198162078857}


 36%|██████████████████████████████▉                                                      | 91/250 [00:00<00:01, 135.82it/s]

...

 96%|████████████████████████████████████████████████████████████████████████████████▉   | 241/250 [00:01<00:00, 143.87it/s]

{'train_loss': 0.3227381408214569}
{'train_loss': 0.27068713307380676}
{'train_loss': 0.27692610025405884}


100%|████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:01<00:00, 136.35it/s]


{'train_loss': 0.2838975191116333}


{'AUPR': 0.047127789703433905,
 'AUPRR': 2.2160510534908364,
 'EP': 676,
 'EPR': 3.748907720246691}